In [9]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Final_Project\Log\feature_selection.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Preprosessed_data\Preprosessed.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df['life_span'] = df['life_span'].abs()

In [3]:
df.isnull().sum()

Unnamed: 0.1          0
Unnamed: 0            0
name                  0
birth_date            0
birth_place           0
death_date            0
death_place           0
occupation            0
awards                0
alma_mater            0
education             0
spouse                0
children              0
occupation_cluster    0
birth_year            0
death_year            0
life_span             0
life_span_cluster     0
edu_award_cluster     0
bio_cluster           0
dtype: int64

In [4]:
import pandas as pd
import plotly.express as px

# Raqamli ustunlarni tanlash
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# faqat raqamli ustunlar bo'yicha korrelyatsiya
corr_matrix = df[numeric_cols].corr()

# life_span ustunining boshqa ustunlar bilan korrelyatsiyasi
life_span_corr = corr_matrix[['life_span']].sort_values(by='life_span', ascending=False)

# Heatmap uchun DataFrame ni transformatsiya qilish
heatmap_data = life_span_corr.reset_index()
heatmap_data.columns = ['Feature', 'Correlation_with_life_span']

# Plotly Express yordamida bar grafik
fig = px.bar(
    heatmap_data,
    x='Feature',
    y='Correlation_with_life_span',
    title='life_span bilan boshqa raqamli ustunlar o\'rtasidagi korrelyatsiya',
    labels={'Correlation_with_life_span': 'Korrelyatsiya qiymati', 'Feature': 'Xususiyatlar'}
)

fig.show()


In [ ]:
import numpy as np
import plotly.figure_factory as ff

# Qiymatlarni 2 xonagacha qisqartirish
annot_text = np.around(corr_matrix.values, decimals=2).astype(str)

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=annot_text,
    colorscale='Viridis',
    showscale=True
)

fig.update_layout(title='Raqamli ustunlar o\'rtasidagi korrelyatsiya matritsasi')
fig.show()

# Feature Selection Filtering 

In [16]:
import seaborn as sns               
import matplotlib.pyplot as plt     


corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  


to_drop = [col for col in upper.columns if any(upper[col] > 0.8)]  
corr_matrix = df.corr().abs() 
print(corr_matrix)  


                    Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0      name  \
Unnamed: 0.2            1.000000      1.000000    1.000000  0.000949   
Unnamed: 0.1            1.000000      1.000000    1.000000  0.000949   
Unnamed: 0              1.000000      1.000000    1.000000  0.000949   
name                    0.000949      0.000949    0.000949  1.000000   
birth_date              0.091798      0.091798    0.091798  0.008020   
birth_place             0.032052      0.032052    0.032052  0.007689   
death_date              0.184592      0.184592    0.184592  0.080515   
death_place             0.144519      0.144519    0.144519  0.119426   
occupation              0.175324      0.175324    0.175324  0.001912   
awards                  0.062499      0.062499    0.062499  0.050228   
alma_mater              0.011318      0.011318    0.011318  0.073215   
education               0.058320      0.058320    0.058320  0.001575   
spouse                  0.012588      0.012588    0.012588  0.00

In [ ]:

import numpy as np   

corr_matrix = df.corr().abs()  

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  

to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]  

df_filtered = df.drop(columns=to_drop)  
print("\nOriginal shape:", df.shape)  

print("Shape after dropping correlated features:", df_filtered.shape) 



Original shape: (2022, 21)
Shape after dropping correlated features: (2022, 19)


In [12]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.2        2022 non-null   int64  
 1   name                2022 non-null   float64
 2   birth_date          2022 non-null   float64
 3   birth_place         2022 non-null   float64
 4   death_date          2022 non-null   float64
 5   death_place         2022 non-null   float64
 6   occupation          2022 non-null   float64
 7   awards              2022 non-null   float64
 8   alma_mater          2022 non-null   float64
 9   education           2022 non-null   float64
 10  spouse              2022 non-null   float64
 11  children            2022 non-null   float64
 12  occupation_cluster  2022 non-null   float64
 13  birth_year          2022 non-null   float64
 14  death_year          2022 non-null   float64
 15  life_span           2022 non-null   float64
 16  life_s

In [17]:
# Yuqori korrelyatsiyaga ega juftliklarni saqlash uchun bo‘sh ro‘yxat yaratamiz
high_corr_pairs = []


for col in upper.columns:
    
    for row in upper.index:
        
        if upper.loc[row, col] is not np.nan and upper.loc[row, col] > 0.95:
            
            high_corr_pairs.append([row, col, round(upper.loc[row, col], 2)])


corr_table = pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

print("Highly correlated feature pairs (correlation > 0.95):")
print(corr_table)

Highly correlated feature pairs (correlation > 0.95):
      Feature 1     Feature 2  Correlation
0  Unnamed: 0.2  Unnamed: 0.1          1.0
1  Unnamed: 0.2    Unnamed: 0          1.0
2  Unnamed: 0.1    Unnamed: 0          1.0


In [19]:
import plotly.express as px  

corr_long = corr_matrix.reset_index().melt(id_vars='index')  

corr_long.columns = ['Feature 1', 'Feature 2', 'Correlation']  

fig = px.imshow(
    corr_matrix,                  
    text_auto='.2f',              
    aspect="auto",                
    color_continuous_scale='RdBu_r',  
    zmin=-1, zmax=1,             
    title="Correlation Matrix Heatmap (Interactive)"  
)


fig.update_layout(
    width=900,          
    height=800,          
    xaxis_title="Features",  
    yaxis_title="Features"   
)

fig.show()

# # Low Variance

In [20]:
# Low Variance
from sklearn.feature_selection import VarianceThreshold  
threshold = 0.01  
selector = VarianceThreshold(threshold=threshold)  
selector.fit(df_filtered)  
low_variance_features = df_filtered.columns[~selector.get_support()] 
df_low_variance_filtered = df_filtered.drop(columns=low_variance_features)  
print("\nOriginal shape:", df_filtered.shape)  
print("Shape after dropping low-variance features:", df_low_variance_filtered.shape)  


Original shape: (2022, 19)
Shape after dropping low-variance features: (2022, 19)


In [21]:
# Past variansli ustunlarni aniqlash:

low_variance_features = df_filtered.columns[~selector.get_support()]
print("Features with low variance (to drop):", list(low_variance_features))

Features with low variance (to drop): []


In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
variances = df[numeric_cols].var()
threshold = 0.01
low_variance_features = variances[variances < threshold].index.tolist()
var_df = pd.DataFrame({
    'Feature': variances.index,
    'Variance': variances.values,
    'LowVariance': ['Yes' if f in low_variance_features else 'No' for f in variances.index]
})


fig = px.bar(
    var_df,
    x='Feature',        
    y='Variance',       
    color='LowVariance',  
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Variance',    
    title='Feature Variance (Low-Variance Highlighted in Red)'  
)


fig.update_layout(
    xaxis_tickangle=-45,
    width=1000,
    height=600
)

fig.show()

In [23]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2022 entries, 0 to 2021
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0.2        2022 non-null   int64  
 1   name                2022 non-null   float64
 2   birth_date          2022 non-null   float64
 3   birth_place         2022 non-null   float64
 4   death_date          2022 non-null   float64
 5   death_place         2022 non-null   float64
 6   occupation          2022 non-null   float64
 7   awards              2022 non-null   float64
 8   alma_mater          2022 non-null   float64
 9   education           2022 non-null   float64
 10  spouse              2022 non-null   float64
 11  children            2022 non-null   float64
 12  occupation_cluster  2022 non-null   float64
 13  birth_year          2022 non-null   float64
 14  death_year          2022 non-null   float64
 15  life_span           2022 non-null   float64
 16  life_s

In [24]:
df_filtered.to_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection\Filtered_Features.csv")

# Feature Selection Wrapper 

 # RFE + Linear Regression

In [25]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

X = df.drop('life_span', axis=1)
y = df['life_span']

In [26]:
# Oddiy chiziqli regressiya modelini yaratamiz (bu model RFE uchun asos bo'ladi)
model = LinearRegression()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)
selected_features = X.columns[rfe.support_]
print("Selected features:", selected_features)

Selected features: Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'name', 'birth_date',
       'birth_place', 'death_date', 'death_place', 'occupation', 'awards',
       'alma_mater', 'education', 'spouse', 'children', 'occupation_cluster',
       'birth_year', 'death_year', 'life_span_cluster', 'edu_award_cluster',
       'bio_cluster'],
      dtype='object')


# RFE + RandomForestRegressor

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

model = RandomForestRegressor()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Selected features (RandomForest):", selected_features)


Selected features (RandomForest): Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'name', 'birth_date',
       'birth_place', 'death_date', 'death_place', 'occupation', 'awards',
       'alma_mater', 'education', 'spouse', 'children', 'occupation_cluster',
       'birth_year', 'death_year', 'life_span_cluster', 'edu_award_cluster',
       'bio_cluster'],
      dtype='object')


# RFE + Decision Tree Regressor 

In [34]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]
print("Selected features (DecisionTree):", selected_features)


Selected features (DecisionTree): Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'name', 'birth_date',
       'birth_place', 'death_date', 'death_place', 'occupation', 'awards',
       'alma_mater', 'education', 'spouse', 'children', 'occupation_cluster',
       'birth_year', 'death_year', 'life_span_cluster', 'edu_award_cluster',
       'bio_cluster'],
      dtype='object')


In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

# Model va ustun sonlarini saqlash uchun bo'sh ro'yxat
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor()    
}

results = []

# X va y aniq ekanligini faraz qilamiz
for name, model in models.items():
    rfe = RFE(model, n_features_to_select=20)  # 20 ta ustun tanlaymiz misol uchun
    rfe.fit(X, y)
    selected_count = sum(rfe.support_)
    results.append({'Model': name, 'Selected Features Count': selected_count})

# Natijalarni DataFrame ko'rinishida chiqaramiz
results_df = pd.DataFrame(results)
print(results_df)


               Model  Selected Features Count
0  Linear Regression                       20
1      Random Forest                       20
2      Decision Tree                       20


In [36]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import os

X = df.drop('life_span', axis=1)
y = df['life_span']

model = LinearRegression()
rfe = RFE(model, n_features_to_select=20)
rfe.fit(X, y)

selected_features = X.columns[rfe.support_]

# Tanlangan ustunlarni DataFrame ga aylantirish
selected_df = pd.DataFrame(selected_features, columns=['Selected Features'])

# Saqlash uchun yo'l
save_path = r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection"
os.makedirs(save_path, exist_ok=True)  # Agar papka mavjud bo'lmasa, yaratadi

# To'liq fayl nomi
file_path = os.path.join(save_path, "Wrapper.csv")

# CSV ga saqlash
selected_df.to_csv(file_path, index=False)

print(f"Tanlangan ustunlar {file_path} ga saqlandi.")


Tanlangan ustunlar C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection\Wrapper.csv ga saqlandi.


# Feature Selection Embedded + Lasso CV

In [38]:
# Xususiyat tanlash uchun Linear Model'larda ishlatiladigan usul (LASSO regression)

# LassoCV (L1-regularizatsiya) ni import qilamiz
# CV - cross-validation, u model uchun optimal alpha (regulyarizatsiya kuchi) tanlab beradi
from sklearn.linear_model import LassoCV
X = df.drop('life_span', axis=1)
y = df['life_span']
lasso = LassoCV(cv=10, random_state=42).fit(X, y)
importance = np.abs(lasso.coef_)
selected_features = X.columns[importance > 0]
print("Selected features using Lasso (non-zero coefficients):")
print(selected_features.tolist())

Selected features using Lasso (non-zero coefficients):
['Unnamed: 0.2', 'birth_place', 'death_date', 'death_place', 'birth_year', 'death_year']


In [ ]:
# Faqat muhim featurelarni saqlab qolish
percentile_threshold = np.percentile(importance, 90) 
top_features = X.columns[importance >= percentile_threshold]
print("\nTop 25% important features based on Lasso coefficients:")
print(top_features.tolist())


Top 25% important features based on Lasso coefficients:
['death_date', 'death_place']


In [42]:
# Feature importance'ni jadval shaklida tayyorlash

feat_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
})

# Importance qiymatlariga qarab kamayish tartibida (eng muhimdan kam muhimga qarab) saralaymiz
feat_df = feat_df.sort_values(by='Importance', ascending=False)
percentile_threshold = np.percentile(importance, 85)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold, 'Yes', 'No')
print("Feature Importance Table:")
print(feat_df)

Feature Importance Table:
               Feature  Importance Top15%
7          death_place    5.816658    Yes
6           death_date    5.063122    Yes
5          birth_place    0.376687    Yes
16          death_year    0.162529     No
15          birth_year    0.141664     No
0         Unnamed: 0.2    0.004554     No
1         Unnamed: 0.1    0.000000     No
2           Unnamed: 0    0.000000     No
3                 name    0.000000     No
9               awards    0.000000     No
8           occupation    0.000000     No
4           birth_date    0.000000     No
11           education    0.000000     No
10          alma_mater    0.000000     No
13            children    0.000000     No
12              spouse    0.000000     No
14  occupation_cluster    0.000000     No
17   life_span_cluster    0.000000     No
18   edu_award_cluster    0.000000     No
19         bio_cluster    0.000000     No


In [44]:
fig = px.bar(
    feat_df,                
    x='Feature',            
    y='Importance',         
    color='Top15%',         
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Importance',      
    title='Lasso Feature Importance'  
)


fig.update_layout(
    xaxis_tickangle=-45,  
    width=1000,           
    height=600            
)

fig.show()

# # Feature Selection Embedded + Random Forest Regression 

In [ ]:
# --- Tree based (Random Forest) Feature Importance ---

# Random Forest Regressorni import qilamiz
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=500, random_state=42)

rf.fit(X, y)

importances = rf.feature_importances_
feat_df = pd.DataFrame({
    'Feature': X.columns,       
    'Importance': importances   
})
feat_df = feat_df.sort_values(by='Importance', ascending=False)
percentile_threshold = np.percentile(importances, 85)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold, 'Yes', 'No')
print("Random Forest Feature Importance Table:")
print(feat_df)

Random Forest Feature Importance Table:
               Feature  Importance Top15%
15          birth_year    0.452131    Yes
16          death_year    0.267030    Yes
17   life_span_cluster    0.085718    Yes
7          death_place    0.066270     No
3                 name    0.037985     No
6           death_date    0.022813     No
5          birth_place    0.012972     No
4           birth_date    0.010566     No
19         bio_cluster    0.009892     No
8           occupation    0.008784     No
2           Unnamed: 0    0.005703     No
1         Unnamed: 0.1    0.005494     No
0         Unnamed: 0.2    0.004796     No
12              spouse    0.003833     No
10          alma_mater    0.002834     No
13            children    0.001082     No
18   edu_award_cluster    0.000789     No
11           education    0.000482     No
14  occupation_cluster    0.000434     No
9               awards    0.000390     No


In [ ]:
# Plotly Express yordamida Random Forest feature importance'ni grafik ko'rinishida chiqaramiz
fig = px.bar(
    feat_df,                
    x='Feature',            
    y='Importance',         
    color='Top15%',         
    color_discrete_map={'Yes': 'red', 'No': 'blue'},  
    text='Importance',      
    title='Random Forest Feature Importance'  
)


fig.update_layout(
    xaxis_tickangle=-45,  
    width=1000,           
    height=600            
)

fig.show()

In [48]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV
import os

X = df.drop('life_span', axis=1)
y = df['life_span']

# LassoCV modeli
lasso = LassoCV(cv=10, random_state=42).fit(X, y)

# Importance qiymatlari (koeffitsientlarning mutlaq qiymati)
importance = np.abs(lasso.coef_)

# Nolga teng bo'lmagan koeffitsientlar bo'yicha tanlangan ustunlar
selected_features = X.columns[importance > 0]

print("Selected features using Lasso (non-zero coefficients):")
print(selected_features.tolist())

# Top 25% muhim ustunlar (percentile orqali)
percentile_threshold = np.percentile(importance, 90)
top_features = X.columns[importance >= percentile_threshold]

print("\nTop 10% important features based on Lasso coefficients:")
print(top_features.tolist())

# Feature importance jadvali yaratish
feat_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
})

# Muqaddaslik bo'yicha kamayish tartibida saralash
feat_df = feat_df.sort_values(by='Importance', ascending=False)

# 85 foizlik percentile chegarasini belgilash
percentile_threshold_85 = np.percentile(importance, 90)
feat_df['Top15%'] = np.where(feat_df['Importance'] >= percentile_threshold_85, 'Yes', 'No')

print("Feature Importance Table:")
print(feat_df)

# CSV saqlash uchun papka yaratish (agar mavjud bo'lmasa)
save_path = r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Feature_selection"
os.makedirs(save_path, exist_ok=True)

# Fayl nomlari
selected_file = os.path.join(save_path, "Lasso_Selected_Features.csv")
importance_file = os.path.join(save_path, "Lasso_Feature_Importance.csv")

# Tanlangan ustunlarni CSV ga saqlash
pd.DataFrame(selected_features, columns=['Selected Features']).to_csv(selected_file, index=False)

# Importance jadvalini CSV ga saqlash
feat_df.to_csv(importance_file, index=False)

print(f"\nTanlangan ustunlar {selected_file} ga saqlandi.")
print(f"Feature importance jadvali {importance_file} ga saqlandi.")


Selected features using Lasso (non-zero coefficients):
['Unnamed: 0.2', 'birth_place', 'death_date', 'death_place', 'birth_year', 'death_year']

Top 10% important features based on Lasso coefficients:
['death_date', 'death_place']
Feature Importance Table:
               Feature  Importance Top15%
7          death_place    5.816658    Yes
6           death_date    5.063122    Yes
5          birth_place    0.376687     No
16          death_year    0.162529     No
15          birth_year    0.141664     No
0         Unnamed: 0.2    0.004554     No
1         Unnamed: 0.1    0.000000     No
2           Unnamed: 0    0.000000     No
3                 name    0.000000     No
9               awards    0.000000     No
8           occupation    0.000000     No
4           birth_date    0.000000     No
11           education    0.000000     No
10          alma_mater    0.000000     No
13            children    0.000000     No
12              spouse    0.000000     No
14  occupation_cluster    0.0